In [12]:
import pandas as pd
import plotly.express as px
import warnings 
import ipywidgets as widgets
from IPython.display import clear_output
import folium
import folium.plugins
warnings.filterwarnings('ignore')

In [13]:
pd.set_option('display.max_columns', None)

In [14]:
df = pd.read_csv('can_scores_quintiles_EN.csv')
df = df.rename(columns={'PRCDDA': 'poi_cbg'})
df['Type'] = 'Urban'
df.loc[df['Dissemination area (DA) Population'] < 400, 'Type'] = 'Rural'
df[df['Dissemination area (DA) Population'] >= 400]
df = df[['poi_cbg', 'Dissemination area (DA) Population', 'Type']]
df

,poi_cbg,Dissemination area (DA) Population,Type
0,10010165,539,Urban
1,10010166,264,Rural
2,10010167,341,Rural
3,10010168,549,Urban
4,10010169,296,Rural
...,...,...,...
54791,59590023,440,Urban
54792,59590024,318,Rural
54793,59590025,683,Urban
54794,59590026,583,Urban


In [15]:
data = pd.read_csv('combined.csv')
# drop rows with Nan values in raw_visit_counts

data = data.dropna(subset=['raw_visit_counts'])
data['mobility_score'] = data['normalized_visits_by_state_scaling'] * data['median_dwell'] / 60

In [16]:
data = data.dropna(subset=['poi_cbg'])
data['poi_cbg'] = (data['poi_cbg'].str[3:]).astype(int)
df1 = data.merge(df, on='poi_cbg')

In [17]:
df1

,Unnamed: 0,placekey,parent_placekey,safegraph_brand_ids,location_name,brands,store_id,top_category,sub_category,naics_code,latitude,longitude,street_address,city,region,postal_code,open_hours,category_tags,opened_on,closed_on,tracking_closed_since,websites,geometry_type,polygon_wkt,polygon_class,enclosed,phone_number,is_synthetic,includes_parking_lot,iso_country_code,wkt_area_sq_meters,date_range_start,date_range_end,raw_visit_counts,raw_visitor_counts,visits_by_day,visits_by_each_hour,poi_cbg,visitor_home_cbgs,visitor_home_aggregation,visitor_daytime_cbgs,visitor_country_of_origin,distance_from_home,median_dwell,bucketed_dwell_times,related_same_day_brand,related_same_week_brand,device_type,normalized_visits_by_state_scaling,normalized_visits_by_region_naics_visits,normalized_visits_by_region_naics_visitors,normalized_visits_by_total_visits,normalized_visits_by_total_visitors,mobility_score,Dissemination area (DA) Population,Type
0,0,zzw-222@65x-9j4-fpv,NaN,NaN,Portugalos Restaurant,NaN,NaN,Restaurants and Other Eating Places,Full-Service Restaurants,722511.0,45.475328,-75.692550,53 Rue Bellehumeur,Gatineau,QC,J8T 6K5,NaN,Portuguese Food,NaN,NaN,2019-07,"[""portugalosrestaurant.ca""]",POLYGON,"POLYGON ((-75.692661298524 45.475306684125286,...",OWNED_POLYGON,False,1.819600e+10,False,False,CA,217.0,2022-05-30T00:00:00-04:00,2022-06-06T00:00:00-04:00,10.0,10.0,"[0,1,4,3,0,1,1]","[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0...",24810116,"{""CA:24810421"":4}","{""CA:24810006"":4,""CA:24830001"":4,""CA:24810008""...","{""CA:24810421"":4}","{""CA"":9}",9859.0,16.5,"{""<5"":0,""5-10"":5,""11-20"":0,""21-60"":4,""61-120"":...","{""Walmart"":30,""Esso"":20,""Ultramar"":20,""Petro-C...","{""Walmart"":30,""Aubainerie"":10,""Esso"":20,""Ultra...","{""android"":9,""ios"":4}",422.662257,0.000078,0.000098,7.420500e-06,1.598488e-05,116.232121,226,Rural
1,0,zzw-222@65x-9j4-fpv,NaN,NaN,Portugalos Restaurant,NaN,NaN,Restaurants and Other Eating Places,Full-Service Restaurants,722511.0,45.475328,-75.692550,53 Rue Bellehumeur,Gatineau,QC,J8T 6K5,NaN,Portuguese Food,NaN,NaN,2019-07,"[""portugalosrestaurant.ca""]",POLYGON,"POLYGON ((-75.692661298524 45.475306684125286,...",OWNED_POLYGON,False,1.819600e+10,False,False,CA,217.0,2022-05-23T00:00:00-04:00,2022-05-30T00:00:00-04:00,17.0,17.0,"[3,1,3,2,4,1,3]","[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1,0,0,0,0,0,0...",24810116,"{""CA:24810426"":4,""CA:24810383"":4,""CA:24810059""...","{""CA:24810025"":4,""CA:24810014"":4,""CA:24810021""...","{""CA:24810059"":4,""CA:24810302"":4,""CA:24810426"":4}","{""CA"":18}",4684.0,8.0,"{""<5"":1,""5-10"":10,""11-20"":2,""21-60"":4,""61-120""...","{""Benny & Co"":6,""Aubainerie"":6,""Ultramar"":6,""S...","{""Walmart"":12,""Benny & Co"":6,""Winners"":6,""Auba...","{""android"":9,""ios"":5}",612.410466,0.000127,0.000159,1.108628e-05,2.135998e-05,81.654729,226,Rural
2,0,zzw-222@65x-9j4-fpv,NaN,NaN,Portugalos Restaurant,NaN,NaN,Restaurants and Other Eating Places,Full-Service Restaurants,722511.0,45.475328,-75.692550,53 Rue Bellehumeur,Gatineau,QC,J8T 6K5,NaN,Portuguese Food,NaN,NaN,2019-07,"[""portugalosrestaurant.ca""]",POLYGON,"POLYGON ((-75.692661298524 45.475306684125286,...",OWNED_POLYGON,False,1.819600e+10,False,False,CA,217.0,2022-05-16T00:00:00-04:00,2022-05-23T00:00:00-04:00,14.0,13.0,"[3,5,1,1,1,1,2]","[0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0...",24810116,"{""CA:24810442"":4,""CA:24810135"":4,""CA:24810428""...","{""CA:24720004"":4}","{""CA:24810060"":4,""CA:24810406"":4,""CA:24810426""...","{""CA"":12}",10057.0,10.0,"{""<5"":0,""5-10"":8,""11-20"":0,""21-60"":2,""61-120"":...","{""Walmart"":15,""Winners"":8,""Esso"":15,""Ultramar""...","{""Walmart"":46,""Benny & Co"":8,""Winners"":8,""Auba...","{""android"":7,""ios"":6}",492.989747,0.000102,0.000129,9.002347e-06,1.607665e-05,82.164958,226,Rural
3,0,zzw-222@65x-9j4-fpv,NaN,NaN,Portugalos Restaurant,NaN,NaN,Restaurants and Other Eating Places,Full-Service Restaurants,722511.0,45.475328,-75.692550,53 Rue Bellehu

In [18]:
def make_map_plot(df_, 
                  plot_limit=15000, 
                  radius_col='normalized_visits_by_state_scaling', 
                  radius_mod=1_000, 
                  color='black', 
                  fill_color='red', 
                  fill_opacity = 0.2, 
                  pop_up_col='normalized_visits_by_state_scaling',
                  tooltip = 'Click for more info.', 
                  zoom_start=4,
                  tiles = 'OpenStreetMap', 
                  map_width = 1400,
                  map_height = 800,
                  marker_type= 'circle', 
                  map_plot=None, 
                  verbose=False):
    # Parameters:
      # df_ is a pandas dataframe. It requires a column called "latitude" and a column called "longitude". 
      # radius_col is a column_name or None. If None, every point is given a fixed radius. 
          # Otherwise, the value in the column radius_col is used as the radius. 
      # radius_mod is to scale your radius units to correspond to units on your map.
      # zoom_start is the scale of the map. Larger numbers = higher resolution.
      # color, fill_color, fill_opacity are marker parameters, see others: https://leafletjs.com/reference-1.3.4.html#path 
      # tiles determines the base layer. Open source options include 'OpenStreetMap', 'Stamen Terrain', 'Stamen Toner'
      # map_width and map_height determine the size of the map image (in pixels)
      # marker_type determines what type of marker is being drawn on the map. Options: 'circle' or 'normal'

    # check valid inputs
    valid_inputs = {'marker_type' : {'val' : marker_type, 'valids' : ['circle', 'normal']},
                    'tiles' :  {'val' : tiles, 'valids': ['OpenStreetMap', 'Stamen Terrain', 'Stamen Toner', 'Mapbox Bright', 'Mapbox Control Room']},
                    'radius_col' : {'val' : radius_col, 'valids' : [None] + [col for col in df_.columns if pd.api.types.is_numeric_dtype(df_[col])]}}
    for param, param_valid_dict in valid_inputs.items():
      if(param_valid_dict['val'] not in param_valid_dict['valids']):
        raise Exception("Invalid parameter input for '{0}'. Valid options are {1}. input value was '{2}' .".format(param, param_valid_dict['valids'], param_valid_dict['val']))   

    # create basemap
    if(not map_plot):
      map_plot = folium.Map(width=map_width,
                            height=map_height,
                            location=[df_.latitude.mean(), df_.longitude.mean()],
                            tiles=tiles,
                            zoom_start=zoom_start,
                            control_scale = True)

    # add markers
    counter = 0
    for index, row in df_.iterrows():
        counter+=1
        if(marker_type=='circle'):
          add_circle_marker_to_map(map_plot, row, radius_col, radius_mod, color, fill_color, fill_opacity, pop_up_col, tooltip)
        elif(marker_type=='normal'):
          add_marker_to_map(map_plot, row, pop_up_col, tooltip)
        if(counter>plot_limit): break
    if(verbose): print("Plotted {0} locations".format(counter))
    return(map_plot)


def add_marker_to_map(map_plot, row, pop_up_col, tooltip):
  folium.Marker([row.latitude, row.longitude],
                        popup= row[pop_up_col] if pop_up_col else None,
                        tooltip=tooltip if pop_up_col else None,
                   ).add_to(map_plot)
  return(None)


def add_circle_marker_to_map(map_plot, row, radius_col, radius_mod, color, fill_color, fill_opacity, pop_up_col, tooltip):
  folium.CircleMarker([row.latitude, row.longitude],
                        radius= row[radius_col]/radius_mod if radius_col else 2,
                        color = color,
                        fill_color = fill_color,
                        weight=0.5,
                        fill_opacity= fill_opacity,
                        popup= row[pop_up_col] if pop_up_col else None,
                        tooltip=tooltip if pop_up_col else None,
                   ).add_to(map_plot)
  return(None)

In [19]:
data['date_range_start'] = data['date_range_start'].str[:10]
data = data.loc[data['date_range_start'] >= '2019-06-01']
data = data.loc[data['date_range_start'] <= '2021-09-01']

In [20]:
options = list(data['date_range_start'].unique())

date_range = widgets.Dropdown(
    options=options,
    value=options[-1],
    description='Date:',
    disabled=False,
)

output = widgets.Output()

In [21]:
# Function that updates plots with slider value

def update_plot(change):
    with output:
        clear_output(wait=True)
        df_ = data[data['date_range_start'] == change.new]
        # df_ = df_[df_['date_range_start'] <= change.new[0]]

        map = make_map_plot(df_)

        title_html = f'''
                <h3 align="center" style="font-size:16px">{change.new}<b></b></h3>
                 '''
        map.get_root().html.add_child(folium.Element(title_html))

        
        map.save(outfile=f'map{change.new}.html')
        display(map)

In [22]:
output = widgets.Output()

with output:
    map = make_map_plot(data[data['date_range_start'] == options[-1]])

    title_html = f'''
            <h3 align="center" style="font-size:16px">{options[-1]}<b></b></h3>
                '''
    map.get_root().html.add_child(folium.Element(title_html))

    display(map)

# Adding slider and reset button in horizontal box
display(widgets.HBox([date_range]))

# Plot update when slider is moved
date_range.observe(update_plot, names='value')

display(output)

Output()